### 一. ORC文件的格式

#### 1. 什么是orc文件
* ORC文件, 全称`Optimized Row Columnar`, 是一种高效存储数据的格式. 他能同时提高数据的read, write, process效率.  
* ORC文件和parquet一样, 不是一个单纯的列式存储结构, 而是首先按照数据行切割整个文件, 单后在行组内部使用列式存储每个字段  
    * 列式文件的好处:  
        * 提高查询效率: 当要查询某一列时, 不用全文见扫描, 可以只读取这一列的数据   
          其次, 文件中会对列加入统计信息: 包括列的max, min, sum等, 因此可以再sql查询时进行RBO中的谓词下推   
        * 由于每一列的数据格式相同, 因此可以采用更高效率的压缩算法进一步减小IO存储  
        * 由于同一列的数据时顺序存储的, 也避免了CPU读取时的缓存缺失

        
#### 2. 格式
* **strips**: orc文件包括很多组的数据行. 这些数据行被称作**strips**; 每个strip包含3部分信息
    1. index data: 索引数据    
       orc文件的strip中包含多个压缩的数据块. 数据块是指定大小或指定行数的row data. orc列式存储的最小单位是一个数据块, 统计信息的最小粒度是一个数据块, 同时形成orc文件时针对多少数据进行一次压缩后形成的也是一个数据块. 默认每隔10000行形成一个数据块, 因此index也是每10000行记录一次  
    2. row data: 真正的数据
    3. strip footer: strip统计信息 
    
    
* **file footer**: 还有一些辅助信息存储在**file footer**中;  
* **post scripts**: 文件的最后部分是**post scripts**, 用来记录文件的元数据(ORC文件是自描述的). 主要包括: 
    1. 文件的meta信息(文件的描述信息)
    2. 所有的strip信息(有多少个strip等等)
    3. 文件的schema信息(列名等)  <img src="img/orc1.jpg">


#### 3. orc中的统计信息, 助力提升where条件查询
orc文件包含统计信息, 比如字段的min, max, count, 是否有空值等信息. 在执行sql语句时助力谓词下推. 比如: min,max助力"<或>"条件. 当进行查询的时候, 如果发现满足查询条件的行并不在统计信息之内, 则可以跳过这个strip, 甚至整个文件.ORC中的统计信息分为3个层次: 
* **file level** : 文件的末尾, postscripts处记录了整个文件中columns的统计信息, 包括: min, max, sum等     
* **stripe level** : 记录该strip的统计信息
* **row level**: 记录strip的row data中这一块数据的统计信息


#### 4. orc文件数据访问方法
* 1. 首先, orc文件的读取是从文件末尾开始的, 一次读入16kb. 文件的最后一个字节(postscript的最后一个字节)记录着PostScript的长度, 改长度不会超过256字节. 有了postscript数据, 就知道了: 
    * 文件的元数据信息, 
    * 数据的压缩格式, 
    * 文件内部每一个压缩快的大小(后续读入数据每次读入一个完整压缩块), 
    * 以及footer部分的长度 
* 2. 接着, 读取footer部分, 确定每个strip的长度和偏移量
* 3. 最后, 根据footer中记录的每个strip位置, 读入strip的index, strip footer信息, 再根据index和footer中的信息确定要读入row data, 每次完整读入一个压缩块


#### 5. 使用布隆过滤器加快strip中的index选择   
前面提到, strip里的统计信息(min,max)可以助力where语句中, 关于"<或>"条件的检索, 避免把不满足条件的strip读入到内存. 那么如果where语句中是"="条件的呢? 这是可以通过让strip对某些列进行布隆过滤器. 


#### 6. [hive使用orc文件存储时的高级参数](https://cwiki.apache.org/confluence/display/Hive/Configuration+Properties#ConfigurationProperties-ORCFileFormat)
<img src="img/paramorc.png" width="85%">

```sql
create table if not exists ${distTable}(
    featureindex string,
    t1_primarykey string, 
    t1_clusterid string, 
    t1_xid string, 
    d1 map<string,string>, 
    t1_id string, 
    t1_zid string, 
    features string, 
    t2_primarykey string, 
    t2_clusterid string, 
    t2_xid string, 
    d2 map<string,string>, 
    t2_id string, 
    t2_zid string, 
    status string
)
partitioned by (save_time string)
STORED AS ORC
tblproperties ("orc.compress"="ZLIB")
```

### 二. Spark读取ORC文件的方式
spark读取orc有3种不同方式, 不同方式的主要区别在于对文件划分split的方式不同(一般不需要修改)
```
原文描述: The HYBRID mode reads the footers for all files if there are fewer files than expected mapper count, switching over to generating 1 split per file if the average file sizes are smaller than the default HDFS blocksize. ETL strategy always reads the ORC footers before generating splits, while the BI strategy generates per-file splits fast without reading any data from HDFS.
```
* **BI** : 
每个文件作为一个split, 这种方式不需要driver节点读取每个orc文件的file footer等任何文件元数据信息  
* **ETL** : driver节点需要读取所有文件的footer信息, 划分文件的几个stripe成为一个split  
* **Hybrid** :默认的读入方式    
    * 如果**文件个数<mapper个数**, 使用ETL模式读取   
    * 如果**表的文件的平均大小<hdfs的块大小**, 则使用BI模式  


```java
val sparkSession = SparkSession
  .builder()
  .appName("PvMvToBase")
  .config("hive.exec.orc.default.stripe.size", 268435456L)
  .config("hive.exec.orc.split.strategy", "BI")  // 可以设置成以BI策略读取orc文件, 减小
  .enableHiveSupport()
  .getOrCreate()
```


如果发生spark任务迟迟无法生成UI界面, 很可能是load orc表数据时, 计算split的时间太长; 此时是driver节点在读取每个文件的footer进行split划分计算, 可以通过将策略设置成BI模式, 跳过计算split的步骤

### 三. Spark-ORC日常操作
#### 1. orc+zlib 和 orc+snappy的对比
ORC+Zlib after the columnar improvements no longer has the historic weaknesses of Zlib, so it is faster than SNAPPY to read, smaller than SNAPPY on disk and only ~10% slower than SNAPPY to write it out.

#### 2. 查看生成的orc文件
* 1. 查看hdfs上的orc文件块信息  
```shell  
$ hdfs fsck hdfs://yq01-ns1/user/hive/warehouse/tmp.db/table1_hive/001319_0 -files -blocks
```
<img src="img/spark-orc-block.png">

* 2. 查看orc文件的stripe个数等信息
```shell
$ hive --orcfiledump hdfs://yq01-ns1/user/hive/warehouse/tmp.db/table1_hive/001319_0 | less
```
<img src="img/spark-orc-stripe.png">

### 四. 控制Spark输出文件的个数   
* 1. **因为每个reducer生成一个orc文件, 所以可以再最后一步带有shuffle的算子上这只partition个数**  
* 2. 设置**spark.sql.shuffle.partitions**, 默认的shuffle并行度, 也能设置生成文件的个数